In [1]:
import numpy as np
from scipy.optimize import minimize

# Parâmetro do problema
P = 20.0
sqrt2 = np.sqrt(2)

def lagrangeano_aumentado(x, mu, u):
    x1, x2 = x
    
    # f(x) = (2*sqrt(2))*x1 + x2
    f = (2*sqrt2)*x1 + x2
    
    g = np.zeros(7)
    # g1: P*(x2 + x1*sqrt2)/(x1^2*sqrt2 + 2*x1*x2) - 20 <= 0
    g[0] = P*(x2 + x1*sqrt2)/(x1**2*sqrt2 + 2*x1*x2) - 20
    # g2: P/(x1 + x2*sqrt2) - 20 <= 0
    g[1] = P/(x1 + x2*sqrt2) - 20
    # g3: 15 - P*x2/(x1^2*sqrt2 + 2*x1*x2) <= 0  =>  P*x2/(...) - 15 >= 0 
    g[2] = 15 - (P*x2/(x1**2*sqrt2 + 2*x1*x2))
    # Limites x1 >= 0.1, x1 <= 5, x2 >= 0.1, x2 <= 5
    g[3] = 0.1 - x1  
    g[4] = x1 - 5
    g[5] = 0.1 - x2
    g[6] = x2 - 5

    return f + np.sum(mu*g) + 0.5*u * np.sum(np.maximum(g, 0)**2)

precisao = 1e-6
max_iter = 100
alpha = 1.5         # Fator de crescimento da penalidade
u = 1.0             # Penalidade inicial
m = 7               # Número de desigualdades
mu = np.zeros(m)    # Multiplicadores iniciais (todos zero)

# Ponto inicial (x1, x2).
x_atual = np.array([1.0, 1.0])
x_ant = np.inf*np.ones_like(x_atual)

# Loop principal
for k in range(max_iter):

    def func_penalizada(x):
        return lagrangeano_aumentado(x, mu, u)
    
    res = minimize(func_penalizada, x_atual, method='Nelder-Mead')
    x_novo = res.x
    
    # Critério de parada: variação relativa de x
    if np.linalg.norm(x_novo - x_ant)/(1 + np.linalg.norm(x_novo)) < precisao:
        x_atual = x_novo
        break
    
    x_ant = x_novo
    x_atual = x_novo
    
    # Atualiza os multiplicadores de Lagrange 
    g_val = np.zeros(m)
    # Recalcular as restrições "no seco" (podemos copiar do L_A ou escrever igual)
    x1, x2 = x_novo
    g_val[0] = P*(x2 + x1*sqrt2)/(x1**2*sqrt2 + 2*x1*x2) - 20
    g_val[1] = P/(x1 + x2*sqrt2) - 20
    g_val[2] = 15 - (P*x2/(x1**2*sqrt2 + 2*x1*x2))
    g_val[3] = 0.1 - x1
    g_val[4] = x1 - 5
    g_val[5] = 0.1 - x2
    g_val[6] = x2 - 5
    
    for i in range(m):
        mu[i] = mu[i] + u * max(g_val[i], -mu[i]/u)
    
    # Atualiza a penalidade
    u *= alpha


print("\n***** RESULTADOS *****")
print("Iterações realizadas:", k+1)
print("x* =", x_atual)
print("f(x*) =", (2*sqrt2)*x_atual[0] + x_atual[1])
print("g(x*):", g_val)
print("mu:", mu)
print("Penalidade final u:", u)



***** RESULTADOS *****
Iterações realizadas: 28
x* = [0.57141712 2.42475291]
f(x*) = 4.040964592422014
g(x*): [ 1.60372122e-05 -1.50006694e+01 -6.85414482e-04 -4.71417119e-01
 -4.42858288e+00 -2.32475291e+00 -2.57524709e+00]
mu: [0.60743752 0.         0.         0.         0.         0.
 0.        ]
Penalidade final u: 56815.128661595285
